In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
# Afternoon tests
start = Time("2023-11-16T20:00:00Z", scale='utc')
end = Time("2023-11-17T10:00:00Z", scale='utc')
#start = Time("2023-07-05 22:20:00Z", scale='utc')
#end = Time("2023-07-05 22:35:00Z", scale='utc')

In [ ]:

state = await client.select_time_series('lsst.sal.ATDome.logevent_mainDoorState', \
                                            ['*'],  start, end)
cmd = await client.select_time_series('lsst.sal.ATDome.logevent_mainDoorCommandedState', \
                                            ['*'],  start, end)
az = await client.select_time_series('lsst.sal.ATDome.logevent_azimuthCommandedState', \
                                            ['*'],  start, end)
shut = await client.select_time_series('lsst.sal.ATDome.position', \
                                            ['mainDoorOpeningPercentage'],  start, end)
close_command = await client.select_time_series('lsst.sal.ATDome.command_closeShutter', \
                                            ['*'],  start, end)

In [ ]:
print(len(state), len(cmd), len(shut), len(close_command))

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,5))
state['state'].plot(ax=ax, color='blue', label='Actual State')
#cmd['commandedState'].plot(ax=ax, color='magenta', ls='--', label='Commanded State')
ax.set_yticks([1,2,3,4,5,6,7])
ax.set_yticklabels(['Closed', 'Opened', 'Ajar', 'Opening', 'Closing','',''])
ax.set_ylabel("Door State")
ax1 = ax.twinx()
shut['mainDoorOpeningPercentage'].plot(ax=ax1, color='red', label='Shutter Open Percentage')
ax1.set_ylabel("Open percentage")
ax1.set_ylim(0,115)
for close in close_command.index:
    ax1.axvline(close, ls='--', color='green', label='Close Command')
ax.legend(loc='lower left')
ax1.legend(loc='upper left')
plt.savefig(f"/home/craiglagegit/DATA/Shutter_State_16Nov23.png")